<a href="https://colab.research.google.com/github/hcpy/Local-GH-Data-for-Python-Project/blob/main/Copy_of_Optional_6b_ML_(SB).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## EXTRA POINTS (5): This is an example of a Machine Learning Pipeline using GenoML. Try to run it on your own

In [13]:
import os
import sys
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
os.chdir("/content/drive/MyDrive/FAES_BIOF309/example_data/discrete")

Mounted at /content/drive/


In [14]:
! mkdir /content/drive/MyDrive/FAES_BIOF309/outputs

mkdir: cannot create directory ‘/content/drive/MyDrive/FAES_BIOF309/outputs’: File exists


In [15]:
! pip install genoml2
! pip install tables --upgrade
! pip install umap-learn
! pip install folium 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
import pandas as pd
pf = pd.read_csv("./validation_pheno.csv")

pf

,ID,PHENO
0,valdiation1,0
1,valdiation2,1
2,valdiation3,1
3,valdiation4,1
4,valdiation5,1
...,...,...
95,valdiation96,1
96,valdiation97,1
97,valdiation98,1
98,valdiation99,1


In [17]:
# Munging

%%bash
genoml discrete supervised munge \
--prefix outputs/test_discrete_geno \
--geno ./validation \
--pheno ./validation_pheno.csv \


---> Running genoml discrete supervised munge

---> ---> Dependency Check

---> ---> ---> Checking plink
---> ---> Dependency Check: [Done]

---> ---> Dependency Check

---> ---> ---> Checking plink
---> ---> Dependency Check: [Done]

Here is some basic info on the command you are about to run.
Python version info...
3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
CLI argument info...
The output prefix for this run is outputs/test_discrete_geno and will be appended to later runs of GenoML.
Working with genotype data? ./validation
Do you want GenoML to prune your SNPs for you? no
The pruning threshold you've chosen is 0.5
Working with additional predictors? None
Where is your phenotype file? ./validation_pheno.csv
Any use for an external set of GWAS summary stats? None
If you plan on using external GWAS summary stats for SNP filtering, we'll only keep SNPs at what P value? 0.001
How strong is your VIF filter? 0
How many iterations of VIF filtering are you doing? 0
The imputation m

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/genoml/__main__.py", line 393, in handle_endpoints
    endpoint(*args)
  File "/usr/local/lib/python3.7/dist-packages/genoml/cli/munging.py", line 75, in main
    df = munger.plink_inputs()
  File "/usr/local/lib/python3.7/dist-packages/genoml/preprocessing/munging.py", line 86, in plink_inputs
    pheno_df.to_hdf(outfile_h5, key='pheno', mode='w')
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py", line 2718, in to_hdf
    encoding=encoding,
  File "/usr/local/lib/python3.7/dist-packages/pandas/io/pytables.py", line 312, in to_hdf
    path_or_buf, mode=mode, complevel=complevel, complib=complib
  File "/usr/local/lib/python3.7/dist-packages/pandas/io/pytables.py", lin

In [ ]:
# Training - running into some errors throughout this notebook
%%bash
genoml discrete supervised train \
--prefix outputs/test_discrete_geno

In [ ]:
# Tuning
%%bash
genoml discrete supervised tune \
--prefix outputs/test_discrete_geno

# New Section

In [ ]:
# Harmonize
%%bash
genoml harmonize \
--test_geno_prefix ./discrete/validation \
--test_prefix outputs/validation_test_discrete_geno \
--ref_model_prefix outputs/test_discrete_geno \
--training_snps_alleles outputs/test_discrete_geno.variants_and_alleles.tab

In [ ]:
#Munge Harmonized Dataset
%%bash
genoml discrete supervised munge \
--prefix outputs/validation_test_discrete_geno \
--geno outputs/validation_test_discrete_geno.refSNPs_andAlleles \
--pheno ./discrete/validation_pheno.csv \
--addit ./discrete/validation_addit.csv \
--ref_cols_harmonize outputs/validation_test_discrete_geno.refColsHarmonize_toKeep.txt

In [ ]:
# Retrain
%%bash
genoml discrete supervised train \
--prefix outputs/test_discrete_geno \
--matching_columns outputs/validation_test_discrete_geno.finalHarmonizedCols_toKeep.txt

In [ ]:
# Test
%%bash
genoml discrete supervised test \
--prefix outputs/validation_test_discrete_geno \
--test_prefix outputs/validation_test_discrete_geno \
--ref_model_prefix outputs/test_discrete_geno.trainedModel